In [1]:
"""
line_extractor_pt2.py 

This script merges redudant 3D lines based on parallel and proximity conditions.
You can tune the paramters defined in helper.py.

Output:
- Rgb images annotated with extracted lines thier semantic labels. 
- Mesh file(.ply) with all merged 3D lines.
- One 3D line Mesh file(.ply) for each semantic label.
- A numpy file containing all the extracted 2D lines and regressed 3D lines.

Author: Haodong JIANG <221049033@link.cuhk.edu.cn>
Version: 1.0
License: MIT
"""
import numpy as np
import os
import open3d as o3d
# from collections import Counter
from joblib import Parallel, delayed
from scipy import stats
import helper

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:

scene_list = ["69e5939669","689fec23d7","c173f62b15","55b2bf8036"]
scene_id = scene_list[2]
### load data path
home = os.path.expanduser("~")
scene_data_path = home+f"/SCORE/line_map_extractor/out/{scene_id}/results_raw.npy"
### result saving path
line_data_folder = home+f"/SCORE/line_map_extractor/out/{scene_id}/" # numpy file with all the extracted 2d lines and merged 3d lines
line_mesh_folder = home+f"/SCORE/line_map_extractor/out/{scene_id}/line_mesh_merged/"
for out_path in [line_data_folder, line_mesh_folder]:
    if not os.path.exists(out_path):
        os.makedirs(out_path)
### load data
scene_data = np.load(scene_data_path, allow_pickle=True).item()
scene_pose = scene_data["scene_pose"]
scene_intrinsic = scene_data["scene_intrinsic"]
obj_id_label_id = scene_data["obj_id_label_id"]
label_2_semantic_dict = scene_data["label_2_semantic_dict"]
scene_line_2d_end_points = scene_data["scene_line_2d_end_points"]
scene_line_2d_semantic_labels = scene_data["scene_line_2d_semantic_labels"]
scene_line_2d_params = scene_data["scene_line_2d_params"]
scene_line_2d_match_idx = scene_data["scene_line_2d_match_idx"]
scene_line_3d_params = scene_data["scene_line_3d_params"]
scene_line_3d_end_points = scene_data["scene_line_3d_end_points"]
scene_line_3d_image_source = scene_data["scene_line_3d_image_source"]
scene_line_3d_semantic_labels = scene_data["scene_line_3d_semantic_labels"]

In [ ]:
### preprocessing for 3d line merging
# each 3D line is treated as a vertex on the graph
# the edges are defined by the parallel and proximity conditions
nnode = len(scene_line_3d_params)
# precomputing to acclearte the graph construction
pi_list = np.array([scene_line_3d_params[i][0].reshape(1, 3) for i in range(nnode)])
vi_list = np.array([scene_line_3d_params[i][1].reshape(1, 3) for i in range(nnode)])
project_null_list = np.eye(3) - np.einsum('ijk,ijl->ikl', vi_list, vi_list)
print("constructing the consistent graph")
def find_neighbors(i):
    edges_i = []
    edges_j = []
    if i % 1000 == 0:
        print("finding neighbors in progress:", i/nnode*100,"%")
    cur_image_idices = [scene_line_3d_image_source[i]] # cur_image_idices stores the indices of image from which the 3D line is extracted
    for j in range(i + 1, nnode):
        if scene_line_3d_image_source[j] not in cur_image_idices: # lines extracted from a same image should not be merged
            if abs(np.dot(vi_list[i], vi_list[j].T)) >= helper.params_3d["parrallel_thresh_3d"]: # parallel condition
                if np.linalg.norm(np.dot(project_null_list[i], (pi_list[i] - pi_list[j]).T)) <= helper.params_3d["overlap_thresh_3d"]: # proximity condition
                    edges_i.append(i)
                    edges_j.append(j)
                    cur_image_idices.append(scene_line_3d_image_source[j])
    return edges_i, edges_j
# parallel processing each 3D line
results = Parallel(n_jobs=helper.params_3d["thread_number"])(delayed(find_neighbors)(i) for i in range(nnode))
## none parallel version
# for i in range(nnode):
#     result = find_neighbors(i)

# organize the edges and store as an itermediate result
edges_i = []
edges_j = []
for edges_i_, edges_j_ in results:
    for i in range(len(edges_i_)):
        i_ = edges_i_[i]
        j_ = edges_j_[i]
    edges_i.extend(edges_i_)
    edges_j.extend(edges_j_)
np.save(line_data_folder+scene_id+"_edges.npy",
        {"edges_i": edges_i, "edges_j": edges_j})

constructing the consistent graph


finding neighbors in progress: 0.0 %


In [6]:
# just for verification
edge_data = np.load(line_data_folder+"edges.npy", allow_pickle=True).item()
nnode = len(scene_line_3d_params)
pi_list = np.array([scene_line_3d_params[i][0].reshape(1, 3) for i in range(nnode)])
vi_list = np.array([scene_line_3d_params[i][1].reshape(1, 3) for i in range(nnode)])
project_null_list = np.eye(3) - np.einsum('ijk,ijl->ikl', vi_list, vi_list)
edges_i = np.array(edge_data["edges_i"])
edges_j = np.array(edge_data["edges_j"])
for idx in range(len(edges_i)):
    i = edges_i[idx]
    j = edges_j[idx]
    if abs(np.dot(vi_list[i], vi_list[j].T)) < helper.params_3d["parrallel_thresh_3d"]:
        print(abs(np.dot(vi_list[i], vi_list[j].T)))
    if np.linalg.norm(np.dot(project_null_list[i], (pi_list[i] - pi_list[j]).T)) > helper.params_3d["overlap_thresh_3d"]:
        print(np.linalg.norm(np.dot(project_null_list[i], (pi_list[i] - pi_list[j]).T)))

In [7]:
# load the saved edges
edge_data = np.load(line_data_folder+"edges.npy", allow_pickle=True).item()
edges_i = np.array(edge_data["edges_i"])
edges_j = np.array(edge_data["edges_j"])
nnode = len(scene_line_3d_params)
# defines the data structure to be stored
merged_scene_line_3d_params=[]
merged_semantic_label_3d =[]
merged_line_3d_image_idx=[]
merged_scene_line_3d_end_points=[]
# starting merging
print("# 3d lines before merging",nnode)
mapping = list(range(0,nnode)) # this lists defines which 3D line a 3D line is merged to.
edges_i_ = np.concatenate((edges_i,np.array(range(0,nnode))))
edges_j_ = np.concatenate((edges_j,np.array(range(0,nnode))))
vertex_concat = np.concatenate((edges_i_,edges_j_))
# if the line is not observed by a enough number of images, we assume it is a background line and discard it
unique_elements, counts = np.unique(vertex_concat, return_counts=True)
vertex_deleted = unique_elements[counts<helper.params_3d["degree_threshold"]+2] # plus 2 because there is an edge between a vertex and itself
# delete the 3d lines
for ver in vertex_deleted:
    mapping[ver]=None
index_deleted = []
for i in range(0,len(edges_i_)):
    if edges_i_[i] in vertex_deleted or edges_j_[i] in vertex_deleted:
        index_deleted.append(i)
edges_i_=np.delete(edges_i_,index_deleted)
edges_j_=np.delete(edges_j_,index_deleted)
countt = 0
while len(edges_i_)>0:  
    # find the vertex with largest degree and merge all its neighbors
    vertex_concat = np.concatenate((edges_i_,edges_j_))
    mode_result = stats.mode(vertex_concat)
    if mode_result.count<helper.params_3d["degree_threshold"]+2: # if the remained lines is not observed by a enough number of images, we assume it is a background line and discard it
        left_vertex = np.unique(vertex_concat)
        for ver in left_vertex:
            mapping[ver]=None
        break
    most_frequent_index = mode_result.mode
    p_3d = scene_line_3d_params[most_frequent_index][0]
    v_3d = scene_line_3d_params[most_frequent_index][1]
    index_1 = np.where(edges_i_==most_frequent_index)
    index_2 = np.where(edges_j_==most_frequent_index)
    # note that the neighbors contain itself 
    neighbors = np.unique(np.concatenate((edges_j_[index_1], edges_i_[index_2]))) 
    # delete vertex nodes
    for neighbor in neighbors:
        mapping[neighbor]=most_frequent_index
        index_1 = np.where(edges_i_==neighbor)
        index_2 = np.where(edges_j_==neighbor)
        index_delete_neighbor = np.unique(np.concatenate((index_1[0], index_2[0])))
        edges_i_=np.delete(edges_i_,index_delete_neighbor)
        edges_j_=np.delete(edges_j_,index_delete_neighbor)
    # update the end points
    end_points = scene_line_3d_end_points[most_frequent_index]
    sig_dim = np.argmax(np.abs(v_3d))
    for neighbor in neighbors:
        end_points_temp = scene_line_3d_end_points[neighbor]
        if end_points_temp[0][sig_dim]<end_points[0][sig_dim]:
            end_points[0] = end_points_temp[0]
        if end_points_temp[1][sig_dim]>end_points[1][sig_dim]:
            end_points[1] = end_points_temp[1]
    # merge 3d lines
    cluster_semantic_labels = np.array(scene_line_3d_semantic_labels[most_frequent_index])
    for neighbor in neighbors:
        cluster_semantic_labels = np.append(cluster_semantic_labels,scene_line_3d_semantic_labels[neighbor])
    unique_cluster_semantic_lables = np.unique(cluster_semantic_labels)
    unique_cluster_semantic_lables = unique_cluster_semantic_lables[unique_cluster_semantic_lables!=0]
    for labels in unique_cluster_semantic_lables:
        merged_scene_line_3d_params.append(scene_line_3d_params[most_frequent_index])
        merged_semantic_label_3d.append(labels)
        merged_line_3d_image_idx.append(scene_line_3d_image_source[most_frequent_index])
        merged_scene_line_3d_end_points.append(end_points)
    if len(unique_cluster_semantic_lables)>3: ### output the 3d line with more than 3 semantic labels
        sample_d = np.linspace(end_points[0][sig_dim], end_points[1][sig_dim], 300)
        point_sets = []
        for d in sample_d:
            point_sets.append(p_3d + (d - p_3d[sig_dim]) / v_3d[sig_dim] * v_3d)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_sets)
        o3d.io.write_point_cloud(line_mesh_folder + f"multiple_semantic_{countt}.ply", pcd)
        countt+=1
        for k in range(0,len(unique_cluster_semantic_lables)):
            print(f"{label_2_semantic_dict[unique_cluster_semantic_lables[k]]},",end="")
print("# 3d lines after merging:",len(merged_scene_line_3d_params))
### calculate projection error after merging 3d lines
print("calculating projection error after merging")
scene_projection_error_r = {}
scene_projection_error_t = {}
for basename in scene_pose.keys():
    projection_error_r = []
    projection_error_t = []
    intrinsic = scene_intrinsic[basename]
    pose_matrix = np.array(scene_pose[basename])
    line_2d_match_idx = np.array(scene_line_2d_match_idx[basename])
    for j in range(0,len(line_2d_match_idx)):
        if line_2d_match_idx[j]==None or mapping[line_2d_match_idx[j]]==None:
            projection_error_r.append(-1)
            projection_error_t.append(-1)
            continue
        mapping_idx = mapping[line_2d_match_idx[j]]
        n_j = scene_line_2d_params[basename][j].reshape(1,3)
        p_3d = scene_line_3d_params[mapping_idx][0].reshape(3,1)
        v_3d = scene_line_3d_params[mapping_idx][1].reshape(3,1)
        n_j_camera = n_j @ intrinsic
        n_j_camera = n_j_camera / np.linalg.norm(n_j_camera)
        n_j_camera = n_j_camera.reshape(3,1)
        error_rot = (pose_matrix[:3, :3] @ n_j_camera).T @ v_3d
        if error_rot>0.1:
            print(basename,error_rot,j,line_2d_match_idx[j],mapping_idx)
        error_trans = (p_3d.T - np.array(pose_matrix)[:3, 3]) @ (pose_matrix[:3, :3] @ n_j_camera)
        projection_error_r.append(np.abs(error_rot))
        projection_error_t.append(np.abs(error_trans))
    scene_projection_error_r[basename] = projection_error_r
    scene_projection_error_t[basename] = projection_error_t

# 3d lines before merging 771
# 3d lines after merging: 90
calculating projection error after merging


In [ ]:
### save data for relocalization experiements
np.save(line_data_folder+scene_id+"_results_merged.npy", {
    "scene_pose": scene_pose,
    "scene_intrinsic": scene_intrinsic,
    "obj_id_label_id": obj_id_label_id,
    "label_2_semantic_dict": label_2_semantic_dict,
    ###
    "scene_line_2d_semantic_labels": scene_line_2d_semantic_labels,
    "scene_line_2d_params": scene_line_2d_params,
    "scene_line_2d_end_points": scene_line_2d_end_points,
    "scene_projection_error_r": scene_projection_error_r,
    "scene_projection_error_t": scene_projection_error_t,
    ###
    "merged_scene_line_3d_params": merged_scene_line_3d_params,
    "merged_scene_line_3d_semantic_labels": merged_semantic_label_3d,
    "merged_scene_line_3d_end_points": merged_scene_line_3d_end_points,
    ###
    "params_3d": helper.params_3d
})

In [ ]:
### output 3d line mesh for visualization 
# save all merged 3d lines
point_sets=[]
for i in range(len(merged_scene_line_3d_params)):
    end_points = merged_scene_line_3d_end_points[i]
    p, v = merged_scene_line_3d_params[i]
    sig_dim = np.argmax(abs(v))
    min_d = end_points[0][sig_dim]
    max_d = end_points[1][sig_dim]
    # get 100 points between min_d and max_d with equal interval
    sample_d = np.linspace(min_d, max_d, 300)
    for d in sample_d:
        point_sets.append(p + (d - p[sig_dim]) / v[sig_dim] * v)
point_sets = np.vstack(point_sets)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_sets)
o3d.io.write_point_cloud(line_mesh_folder+scene_id+f"_merged_3d_line_mesh.ply", pcd)

# save the 3d line mesh for each semantic label
def process_label(i, semantic_label):
    if int(semantic_label) == 0:
        return
    index = np.where(merged_semantic_label_3d == semantic_label)
    print("semantic label:" + f"{label_2_semantic_dict[int(semantic_label)]}" + " number of lines:", len(index[0]))
    point_sets = []
    for j in range(len(index[0])):
        end_points = merged_scene_line_3d_end_points[index[0][j]]
        p, v = merged_scene_line_3d_params[index[0][j]]
        sig_dim = np.argmax(abs(v))
        min_d = end_points[0][sig_dim]
        max_d = end_points[1][sig_dim]
        # get 300 points between min_d and max_d with equal interval
        sample_d = np.linspace(min_d, max_d, 300)
        for d in sample_d:
            point_sets.append(p + (d - p[sig_dim]) / v[sig_dim] * v)
    if point_sets:
        point_sets = np.vstack(point_sets)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_sets)
        o3d.io.write_point_cloud(line_mesh_folder + f"{label_2_semantic_dict[int(semantic_label)]}.ply", pcd)
semantic_labels_all = np.unique(merged_semantic_label_3d)
# Parallel(n_jobs=params_3d["thread_number"])(delayed(process_label)(i, semantic_label) for i, semantic_label in enumerate(semantic_labels_all))
Parallel(n_jobs=8)(delayed(process_label)(i, semantic_label) for i, semantic_label in enumerate(semantic_labels_all))



semantic label:floor number of lines: 27
semantic label:wall number of lines: 20
semantic label:chair number of lines: 8
semantic label:sofa number of lines: 5
semantic label:table number of lines: 4
semantic label:whiteboard number of lines: 7
semantic label:heater number of lines: 5
semantic label:ladder number of lines: 5
semantic label:projector screen number of lines: 6
semantic label:pillow number of lines: 1
semantic label:ceiling number of lines: 1
semantic label:speaker number of lines: 1


[None, None, None, None, None, None, None, None, None, None, None, None]